# Nome: Matheus da Silva Oliveira
# DRE: 118178020

## 1

Será usada a [biblioteca Test](https://docs.julialang.org/en/v1/stdlib/Test/) para facilitar nos units tests.

In [2]:
using LinearAlgebra
using Test

### RESOLVE_DIAGONAL

Como sabemos, quando a matriz $A$ é diagonal, o sistema $Ax=b$ é determinado, pois todos os $x_i$ são determinados por apenas uma divisão, desde que a diagonal de $A$ seja não nula. Caso fosse, deveriámos olhar para o vetor $b$, mas não precisamos nos preocupar com este caso agora. Sendo assim, determinamos todos os elementos do vetor $x$ através da fórmula
$$
x_i = \frac{b_i}{a_{ii}}
$$

In [6]:
"""
    resolve_diagonal(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear diagonal Ax=b.
A é uma matriz nxn e b nx1.
"""
function resolve_diagonal(A, b)
    # Computa n, que será o número de linhas da matriz x
    n = length(b)
    x = zeros(n)
    # Computa o vetor x
    for i=1:n
        x[i] = b[i] / A[i,i]
    end
    return x
end

resolve_diagonal

#### Testes

As matrizes para os testes são geradas randomicamente. A matriz identidade $I$ concedida pela biblioteca *LinearAlgebra* foi usada para gerar as matrizes diagonais.

In [8]:
# n é o número de linhas e colunas da matriz A_i e o número de linhas da matriz b_i
n = 5
A1 = randn()*Matrix{Float64}(I,n,n)
b1 = randn(n)

n = 6
A2 = randn()*Matrix{Float64}(I,n,n)
b2 = randn(n)

n = 2
A3 = randn()*Matrix{Float64}(I,n,n)
b3 = randn(n)

2-element Vector{Float64}:
 0.6958678176188422
 1.0964548754524037

In [9]:
x1 = resolve_diagonal(A1,b1)
@test A1*x1 ≈ b1 

Test Passed

In [10]:
x2 = resolve_diagonal(A2,b2)
@test A2*x2 ≈ b2 

Test Passed

In [12]:
x3 = resolve_diagonal(A3,b3)
@test A3*x3 ≈ b3 

Test Passed

### RESOLVE_TRIANGULAR_SUPERIOR

Como visto nos vídeos do Abel, a fórmula para cada $x_i$ é dado por 
$$
x_i = \frac{b_i-\sum_{j=i+1}^{n} a_{ij}x_j}{a_{ii}}
$$
onde $a_{ij}$ são os elementos da matriz $A$.

In [26]:
"""
    resolve_triangular_superior(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear Ax=b, onde
A é uma matriz nxn triangular superior e b possui tamanho nx1.
"""
function resolve_triangular_superior(A, b)
    # Computa n, que será o número de linhas da matriz x
    n = length(b)
    x = zeros(n)
    #= Para cada linha i de A e b, começando na última linha e terminando na primeira,
    computamos cada variável nas colunas por substituição direta=# 
    for i = n:-1:1
        x[i] = b[i]
        for j = i+1:n
            x[i] = x[i] - (A[i,j]*x[j])
        end
        x[i] = x[i]/A[i,i]
    end
    return x
end

resolve_triangular_superior

#### Testes

Como o $det(A) \neq 0$, sendo A uma matriz triangular superior ou inferior, então o sistema é determinado, visto que o determinante de uma matriz triangular superior ou inferior é o produto de sua diagonal principal. Sendo assim, utilizamos a função [UpperTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.UpperTriangular) para gerar as matrizes $A_i$. 

In [59]:
n = 5
A1 = randn(n,n)
A1 = UpperTriangular(A1)
b1 = randn(n)

n = 6
A2 = randn(n,n)
A2 = UpperTriangular(A2)
b2 = randn(n)

n = 2
A3 = randn(n,n)
A3 = UpperTriangular(A3)
b3 = randn(n)

2-element Vector{Float64}:
  1.3822534864866032
 -0.9166586582046224

In [63]:
x1 = resolve_triangular_superior(A1,b1)
@test A1*x1 ≈ b1 

Test Passed

In [64]:
x2 = resolve_triangular_superior(A2,b2)
@test A2*x2 ≈ b2 

Test Passed

In [65]:
x3 = resolve_triangular_superior(A3,b3)
@test A3*x3 ≈ b3 

Test Passed

### RESOLVE_TRIANGULAR_INFERIOR

In [27]:
function resolve_triangular_inferior(A, b)
    n = length(b)
    x = zeros(n)
    for i = 1:n
        # Vetor com os resultados
        x[i] = b[i]
        for j = 1:(i - 1)
            x[i] = x[i] - (A[i,j]*x[j])
        end
        x[i] = x[i]/A[i,i]
    end
    return x
end

resolve_triangular_inferior (generic function with 1 method)

In [28]:
function decomposicao_LU(A)
    # Pega o número de linhas de A
    n, = size(A)
    # Gera a matriz que será L 
    L = Matrix{Float64}(I,n,n)
    # Fazemos uma cópia de A. Será nossa matriz triangular superior
    U = copy(A)
    # Constante que multiplica as linhas
    v = 0
    for i = 1:(n-1)
        #j = i+1
        #linha = argmax(abs.A[j:n,j])
        #U[[j,linha],:] = [[linha,j],:]
        pivo = U[i,i]
        # Para cada linha abaixo da linha do pivo, realizamos o escalonamento
        for j = (i+1):n 
            # Constante que multiplicará toda uma linha
            v = U[j,i]/pivo
            # Multiplica todos elementos da linha correspondente para balancear o sistema
            for k = 1:n  
                U[j,k] = U[j,k] - U[i,k]*v
            end
            # Zeramos os elementos manualmente devido ao erro de precisão da máquina
            U[j,i] = 0 
            # Gera a matriz triangular inferior elemento por elemento
            L[j,i] = v
        end
    end
    return L,U
end         

decomposicao_LU (generic function with 1 method)

In [29]:
A =37*randn(5,5)
L,U = decomposicao_LU(A)
L*U ≈ A 
#U
L

5×5 Matrix{Float64}:
  1.0         0.0        0.0       0.0       0.0
 -0.454159    1.0        0.0       0.0       0.0
 -0.371855    0.38642    1.0       0.0       0.0
  0.0473934  -0.605284  -0.821462  1.0       0.0
 -0.592716   -0.224049   1.2762    0.560414  1.0

In [30]:
A =37*randn(5,5)

println(A[:,1])
println(A)

[19.854281793592072, -29.004165448492728, 15.617598835025177, -7.112150631822241, -2.105591565816797]
[19.854281793592072 -17.79417334016122 25.448920800277424 40.23602409232503 -23.24155539242017; -29.004165448492728 -45.10718112436474 9.30145766063689 43.28304227534812 14.886010439336818; 15.617598835025177 8.680475604395818 18.62876948399793 -2.1257932735222065 -40.62958957414308; -7.112150631822241 41.45350745668476 -50.490287907772185 65.98136157428047 52.941718104067; -2.105591565816797 -48.63404695474461 1.106432573470475 -8.058803320976862 33.56061462809958]


In [31]:
function resolve(A,b)
    L,U = decomposicao_LU(A)
    Y = resolve_triangular_inferior(L,b)
    return resolve_triangular_superior(U,Y)
end

resolve (generic function with 1 method)

In [32]:
A = 37*randn(n,n)
b = A * ones(n)
@test resolve(A,b) ≈ A\b

Test Passed

In [33]:
A = randn(n,n)

5×5 Matrix{Float64}:
  0.980845  -1.51729   -0.582003   0.17536  -0.920429
  1.4635    -0.335334   0.119629   1.02202   1.20501
  1.99921    0.139928  -2.11274    1.60903  -0.489038
 -2.60808   -0.30748    0.214922  -1.99028   0.469373
  1.33919   -0.672299  -1.48995    0.19624   0.605214

In [34]:
A[:,1] = A[:,5]
A

5×5 Matrix{Float64}:
 -0.920429  -1.51729   -0.582003   0.17536  -0.920429
  1.20501   -0.335334   0.119629   1.02202   1.20501
 -0.489038   0.139928  -2.11274    1.60903  -0.489038
  0.469373  -0.30748    0.214922  -1.99028   0.469373
  0.605214  -0.672299  -1.48995    0.19624   0.605214

In [35]:
function inversa(A)
    n, = size(A)
    A_inv = zeros(n, n)
    Ident = Matrix{Float64}(I,n,n)
    for i=1:n
        A_inv[:,i] = resolve(A,Ident[:,i])
    end
    return A_inv
end               

inversa (generic function with 1 method)

In [36]:
A = randn(n,n)
@test inversa(A) ≈ inv(A)

Test Passed

## 3

In [ ]:
function discretiza(n)
    A = zeros(n,n)
    A[:,1] .= 25
    A[1,:] .= 20 
    A[:,n] .= 20 
    A[n,:] .= 30
    for i=1:(n-1)
        for j=1:(n-1)
            
        end
    end
end

In [ ]:
function discretiza(n)
    A = zeros(n)
    b = zeros(n)
    for i=1:n^2
        A[i,i] = 1
        if i <= n 
            b[i] += 20/4
        end
        if (i-1)%n == 0
            b[i] += 25/4
        end
        if i%n == 0
            b[i] += 20/4
        end
        if i >= n*(n-1) + 1
            b[i] += 30/4
        end
    end
end

0

In [80]:
n = 7
 A = zeros(n,n)
A[:,1] .= 25
A[1,:] .= 20 
A[:,n] .= 20 
A[n,:] .= 30
A

7×7 Matrix{Float64}:
 20.0  20.0  20.0  20.0  20.0  20.0  20.0
 25.0   0.0   0.0   0.0   0.0   0.0  20.0
 25.0   0.0   0.0   0.0   0.0   0.0  20.0
 25.0   0.0   0.0   0.0   0.0   0.0  20.0
 25.0   0.0   0.0   0.0   0.0   0.0  20.0
 25.0   0.0   0.0   0.0   0.0   0.0  20.0
 30.0  30.0  30.0  30.0  30.0  30.0  30.0

## 4

In [76]:
A = [
    1 0 0 0 0 0 0 0;
    0 1 0 0 0 0 0 0;
    0 0 1 0 0 0 0 0;
    0 -1 0 -1 0 1 0 0;
    0 0 0 0 -1 0 1 0;
    0 0 0 0 0 1 -1 1;
    -1 0 0 1 0 0 0 0;
    0 0 -1 0 1 0 0 0;
   ]

b = [
    2000+5000;
    1500+2000;
    8000+1000;
    0;
    3000;
    500; 
    30000;
    3000;  
]

resolve(A,b)

8-element Vector{Float64}:
   7000.0
   3500.0
   9000.0
  37000.0
  12000.0
  40500.0
  15000.0
 -25000.0

In [75]:
A1 = hcat(A,[0; 0; 0; 0; 0; 0; -1; 1])
resolve(A1,b)

8-element Vector{Float64}:
   7000.0
   3500.0
   9000.0
  37000.0
  12000.0
  40500.0
  15000.0
 -25000.0

Estamos tratando apenas matrizes $A_{nxn}$, mas $A1 = A_{8x9}$